$\large\textbf{Exercise 2.}$

$\large\textbf{Consider the function :}$ \\
\begin{align}
\mathbf{q(x)} = \sqrt{x_1^2 + 1} + \sqrt{x_2^2 + 1} 
  \\
\end{align}

$\large\textbf{1.}$ Write code for implementing all relevant modules necessary for Newton’s method and gradient descent method
to solve $min_{x} q(x).$

In [55]:
import numpy as np 
import scipy.linalg as lg 

#method to find Hessian matrix
def evalh(x):
  assert type(x) is np.ndarray 
  assert len(x) == 2

  return np.array([[1*((x[0]**2+1)**(-3/2)),0],[0,1*((x[1]**2+1)**(-3/2))]])

#method to find the condition number of any square matrix
import scipy.linalg as la
def find_condition_number(A):
  assert type(A) is np.ndarray
  assert A.shape[0] == A.shape[1]
  min_eigenval = min(np.linalg.eig(A)[0])
  max_eigenval = max(np.linalg.eig(A)[0])
  return max_eigenval/min_eigenval

In [56]:
def compute_D_k(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  #compute and return D_k
  A1=np.array([[1*((x[0]**2+1)**(-3/2)),0],[0,1*((x[1]**2+1)**(-3/2))]])
  return np.linalg.inv(A1)

In [57]:
def evalf(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2  
  return (x[0]**2+1)**(1/2)+(x[1]**2+1)**(1/2)

In [58]:
def evalg(x):  
  assert type(x) is np.ndarray and len(x) == 2 
  return np.array([x[0]/((x[0]**2+1)**(1/2)),x[1]/((x[1]**2+1)**(1/2))])

In [59]:
def compute_steplength_backtracking(x, gradf, alpha_1, rho, gamma): 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_1) is float and alpha_1>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  
  while evalf(x+alpha_1*(-gradf)) > evalf(x)-gamma*alpha_1*np.dot((gradf.T),gradf):
    alpha_1=rho*alpha_1
  return alpha_1

In [60]:
def compute_steplength_backtracking_scaled_direction(x, gradf, direction, alpha_1, rho, gamma): 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(direction) is np.ndarray and len(direction) == 2 
  assert type(alpha_1) is float and alpha_1>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0.
  
  while evalf(x+alpha_1*direction) > evalf(x)+gamma*alpha_1*np.matmul(direction,np.matrix.transpose(gradf)):
    alpha_1=rho*alpha_1 

  return alpha_1

In [61]:
#line search type 
Backtracking_Line_Search = 1
Constant_Step_Length = 2

In [62]:
def find_minimizer_gd(start_x, tol, line_search_type,*args):
  assert type(start_x) is np.ndarray and len(start_x) == 2 
  assert type(tol) is float and tol>=0 

  x = start_x
  g_x = evalg(x)
  #initialization for backtracking line search 
  if (line_search_type == Backtracking_Line_Search):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    #print('Params for Backtracking LS: direction:', direction, alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)
  
  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): 
    
  
    if line_search_type == Backtracking_Line_Search:
      step_length = compute_steplength_backtracking(x,g_x,alpha_start,rho, gamma) 
    
      #raise ValueError('Backtracking Line Search NOT YET IMPLEMENTED')
    elif line_search_type == Constant_Step_Length: 
      step_length = 0.1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    A=evalh(x)
    #implement the gradient descent steps here   
    x = np.subtract(x,(step_length*g_x))
    k = k+1 
    g_x = evalg(x) 


    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x, k,evalf(x)

$\large\textbf{2.[R]}$ Consider $\eta$^k =1  , $\forall$  k= 1, 2, 3, . . . in algorithm 1. With starting point $x^0$ =(1,1) and a stopping tolerance
$\tau$ = $10^{−9}$ , find the number of iterations taken by the Newton’s method. Compare the number of iterations with that taken by Newton’s method (with backtracking line search) in Algorithm 1. Note the minimizer and minimum objective function value in each case. Comment on your observations

In [63]:
def find_minimizer_gdscaling(start_x, tol, line_search_type,*args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2  
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x)
  k=0
 
  if (line_search_type == Backtracking_Line_Search):
    alpha_1 = args[0]
    rho = args[1]
    gamma = args[2]
    
  
  
  while (np.linalg.norm(g_x) > tol):

    direction = np.matmul(compute_D_k(x),-g_x)
    if line_search_type == Constant_Step_Length:
      step_length=1.0
    elif line_search_type == Backtracking_Line_Search:
      step_length = compute_steplength_backtracking_scaled_direction(x,g_x,direction,alpha_1,rho, gamma)

    
    
    #implement the gradient descent steps here   
    x = np.add(x,step_length*direction) 
    k = k+1 
    g_x = evalg(x) 
    gradf = evalf(x)
    #print('condition_number',A,'iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x,k,evalf(x)  

In [64]:
#check gradient descent with constant step length 
my_start_x = np.array([1.0,1.0])
my_tol= 1e-9

x_opt = find_minimizer_gdscaling(my_start_x, my_tol, Constant_Step_Length)
print(x_opt)

(array([3.93389978e-12, 3.93389978e-12]), 36, 2.0)


In [65]:
#check gradient descent with backtracking line search 

x_opt_blsg = find_minimizer_gdscaling(my_start_x, my_tol, Backtracking_Line_Search ,1.0, 0.5,0.5)
print(x_opt_blsg)

(array([2.22044605e-16, 2.22044605e-16]), 1, 2.0)


We can clearly see that  by Newtons method: 

 By backtracking line Search Total no.of iterationssearch = 1   .    
Minimizer of the function =([2.22044605e-16, 2.22044605e-16])      
function value = 2.0

And    
 Total no of iteration  in  constant step length = 36    
 Minimizer of the function =([3.93389978e-12, 3.93389978e-12])    
function value = 2.0


$\large\textbf{3.[R]}$ Compare the number of iterations obtained for the two variants of Newton’s method in the previous question, with that of gradient descent algorithm (without scaling) with backtracking line search, gradient descent algorithm (without scaling ) with backtracking line search (implemented in previous
labs), with starting point (1, 1). For backtracking line search, use $\alpha^0$ = 1, $\rho$ = 0.5, $\gamma$ = 0.5. Also compare the
minimizer and minimum objective function value in each case. Comment on your observations.

In [66]:
my_start_x = np.array([1.0,1.0])
my_tol= 1e-9
x_opt = find_minimizer_gd(my_start_x, my_tol, Backtracking_Line_Search, 1.0, 0.5,0.5)
print('No. of iterations:',x_opt[1],'Minimizer of the function :',x_opt[0],'Function value:',x_opt[2])

No. of iterations: 4 Minimizer of the function : [2.78991477e-19 2.78991477e-19] Function value: 2.0


No. of iterations: 4                      
Minimizer of the function : [2.78991477e-19 2.78991477e-19]  
 Function value: 2.0


Hence ,when we apply newtons method with this starting point backtracking since that takes less iterations than without scaling backtracking approach while without scaling does better than newtons method with constant step length.



$\large\textbf{4.[R]}$ Consider $\eta$^k =1  , $\forall$  k= 1, 2, 3, . . . in algorithm 1. With starting point $x^0$ =(10,10) and a stopping tolerance
$\tau$ = $10^{−9}$ , find the number of iterations taken by the Newton’s method. Compare the number of iterations with that taken by Newton’s method (with backtracking line search) in Algorithm 1. Note the minimizer and minimum objective function value in each case. Comment on your observations

In [67]:
def find_minimizer_gdscaling(start_x, tol, line_search_type,*args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2  
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x)
  k=0
 
  if (line_search_type == Backtracking_Line_Search):
    alpha_1 = args[0]
    rho = args[1]
    gamma = args[2]
    
  
  
  while (np.linalg.norm(g_x) > tol):

    direction = np.matmul(compute_D_k(x),-g_x)
    if line_search_type == Constant_Step_Length:
      step_length=1.0
    elif line_search_type == Backtracking_Line_Search:
      step_length = compute_steplength_backtracking_scaled_direction(x,g_x,direction,alpha_1,rho, gamma)

    
    
    #implement the gradient descent steps here   
    x = np.add(x,step_length*direction) 
    k = k+1 
    g_x = evalg(x) 
    gradf = evalf(x)
    #print('condition_number',A,'iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x,k,evalf(x) 

In [68]:
#check gradient descent with constant step length 
my_start_x = np.array([10.0,10.0])
my_tol= 1e-9

x_opt = find_minimizer_gdscaling(my_start_x, my_tol, Constant_Step_Length)
print(x_opt)

(array([-1.e+243, -1.e+243]), 5, inf)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in double_scalars
  after removing the cwd from sys.path.


In [69]:
#check gradient descent with backtracking line search 

x_opt_blsg = find_minimizer_gdscaling(my_start_x, my_tol, Backtracking_Line_Search ,1.0, 0.5,0.5)
print(x_opt_blsg)

(array([-9.92761578e-15, -9.92761578e-15]), 17, 2.0)




By backtracking line Search Total no.of iterationssearch = 17 .   
Minimizer of the function =([-9.92761578e-15, -9.92761578e-15])    
function value = 2.0

And    
By Constant step length solution is not obtained  by newtons method.

$\large\textbf{5.[R]}$ Compare the number of iterations obtained for the two variants of Newton’s method in the previous question, with that of gradient descent algorithm (without scaling) with backtracking line search, gradient descent algorithm (without scaling ) with backtracking line search (implemented in previous
labs), with starting point (10, 10). For backtracking line search, use $\alpha^0$ = 1, $\rho$ = 0.5, $\gamma$ = 0.5. Also compare the
minimizer and minimum objective function value in each case. Comment on your observations.

In [71]:
my_start_x1 = np.array([10.0,10.0])
my_tol= 1e-9
x_opt_gd = find_minimizer_gd(my_start_x1, my_tol, Backtracking_Line_Search, 1.0, 0.5,0.5)
print('No. of iterations:',x_opt_gd[0],'Minimizer of the function:',x_opt_gd[1],'Function value:',x_opt_gd[2])

No. of iterations: [2.12455853e-14 2.12455853e-14] Minimizer of the function: 13 Function value: 2.0


Now here newtons methods takes more iterations so clearly it depens on the starting point that wich approach will be best for a given function so here is reason that starting point is also one of the factors to give the best approach.